In [ ]:
# Markov Attempt 4 
# Changelog: 
# o Uploaded the fuction for updating the bad words. 
# o Uploaded a new function for recursively importing the cleaned lyrics. This way it's always updated.  
# All lyrics - way more lines made - same rhyme ranker as in attempt 1 

In [35]:
!pip install PyGithub

# Package Imports
import random
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt 
from urllib.request import urlopen # The default requests package
import requests # For making GitHub requests
from pprint import pprint # For pretty printing
from pathlib import Path # The Path class

# For the more advanced requests
import base64
import os
import sys
sys.path.append("./PyGithub");
from github import Github
from getpass import getpass

In [42]:
# Recursively Import the Data (AUTOMATIC)

def import_github(path_name="AllLyrics.txt"):
  """
  Function for importing the github file
  path_name: str
  output: None
  """
  g = Github(getpass("Enter your PAT key ")) # Enter your PAT Key.
  username = "MikeMNelhams"
  main_branch_bool = input("Main Branch: Yes or No? ")
  yes_synonyms = ["yes", "y", "yh", "1", "true"]
  if main_branch_bool.lower() in yes_synonyms: 
    branch = "master" 
  else: 
    branch = "PROTOTYPE"

  user = g.get_user(username)
  for repo in g.get_user().get_repos():
      if repo.name == "ai-group-project-Team-JMJM":
        r_proj_clone = repo
        break
      # To see all the available attributes and methods
      print(dir(repo))

  print("Importing Github cleaned text files...")
  contents = repo.get_contents("RapLyrics/CLEAN", ref=branch)

  for file_ in contents:
    path = file_.path
    path = str(path) 
    # Only choose the .txt files
    if path[-4:] == '.txt':
      # Append the Lyrics
      RAP_DATA.append(file_.decoded_content.decode("utf-8")) 
  
  temp_path = Path(path_name)
  if temp_path.is_file():
    with open(path_name, 'r') as temp_read: 
      if os.stat(path_name).st_size == 0:
        write_bool2 = True
      else: 
        write_bool2 = False
  else: 
    write_bool2 = True
  
  if write_bool2: 
    for lyric in RAP_DATA: 
      try:
         with open(path_name, 'w') as writefile: 
          writefile.write(lyric)
      except: 
        print("Error, file moved/deleted during write")
    print("{} is now up to date!".format(path_name))
  else: 
    print("{} is already up to date!".format(path_name))


def update_github(write_bool=False, path_name="AllLyrics.txt"):
  """
  Function for updating the github file, by cleaning the lyrics, optional write to txt file. 
  write_bool: bool
  path_name: str
  output: None
  """
  g = Github(getpass("Enter your PAT key ")) # Enter your PAT Key.
  username = "MikeMNelhams"
  main_branch_bool = input("Main Branch: Yes or No? ")
  yes_synonyms = ["yes", "y", "yh", "1", "true"]
  if main_branch_bool.lower() in yes_synonyms: 
    branch = "master" 
  else: 
    branch = "PROTOTYPE"

  user = g.get_user(username)
  for repo in g.get_user().get_repos():
      if repo.name == "ai-group-project-Team-JMJM":
        r_proj_clone = repo
        break
      # To see all the available attributes and methods
      print(dir(repo))

  print("Importing Github uncleaned text files...")
  contents = repo.get_contents("RapLyrics/UNCLEAN", ref=branch)

  RAP_DATA = []
  rap_lyric_names = []

  for file_ in contents:
    path = file_.path
    path = str(path) 
    # Only choose the .txt files
    if path[-4:] == '.txt':
      # Append the name
      title_start = path.find('UNCLEAN')
      title_len = path[title_start:].find('.')
      name = path[title_start + 8:title_start + title_len]
      if name[-2:] == 'UC':
        name = name[:-2]
      rap_lyric_names.append(name) 

      # Append the Lyrics
      RAP_DATA.append(file_.decoded_content.decode("utf-8")) 
    elif path == 'RapLyrics/UNCLEAN/censors.csv':
      censors = file_.decoded_content

      # Split the long string into a list of lines, then split by words, then put into a csv, then to numpy arr
      data = censors.decode('utf-8')[1:].splitlines()
      data_rows = []
      for count, word in enumerate(data):
        if count>0:
          data_rows.append(word.split(','))
      censors = pd.DataFrame(data_rows)
      censors = censors.to_numpy()
      
  # Remove the \ufeff at the beginning O(n)
  for count, lyric in enumerate(RAP_DATA): 
    RAP_DATA[count] = lyric[1:]

  # Censor the profanities O(n*m)
  for count in range(len(RAP_DATA)): 
    for i in range(len(censors[0:])):
      RAP_DATA[count] = RAP_DATA[count].replace(str(censors[i, 0]), str(censors[i, 1]))

  contents = repo.get_contents("RapLyrics/CLEAN", ref=branch)
  cleaned_names = []
  for counter, file_ in enumerate(contents):
    path = file_.path
    path = str(path) 
    print("File {} ".format(counter + 1) + path)
    # Only choose the .txt files
    if path[-4:] == '.txt':
      # Append the name
      title_start = path.find('CLEAN')
      title_len = path[title_start:].find('.')
      name = path[title_start + 6:title_start + title_len]
      if name[-2:] == 'CL':
        name = name[:-2]
      cleaned_names.append(name) 

  # ALL OF THE EDITING IS DONE IN THE 'PROTOTYPE BRANCH' to avoid overwriting import changes
  # If the (now cleaned) rap_lyrics name is new (not in cleaned_names), then we want to create that as a new file 
  # If the (now cleaned) rap_lyrics name is NOT new (not in cleaned_names), then we want to update the file
  # print(rap_lyric_names)
  # print(cleaned_names)
  print("Commiting files to github...")
  for counter, new_name in enumerate(rap_lyric_names): 
    if new_name in cleaned_names: 
      duplicate = repo.get_contents("RapLyrics/CLEAN/{}CL.txt".format(new_name), ref=branch)
      repo.update_file("RapLyrics/CLEAN/{}CL.txt".format(new_name), "This was uploaded automatically via pipeline", RAP_DATA[counter], duplicate.sha, branch=branch)
    else:
      repo.create_file("RapLyrics/CLEAN/{}CL.txt".format(new_name), "This was uploaded automatically via pipeline", RAP_DATA[counter], branch=branch)

  if write_bool: 
    print("Writing text file to: {}".format(path_name))
    with open(path_name, 'w') as writefile:
      for lyric in RAP_DATA:
        writefile.write(lyric)


In [ ]:
# Import all of Mike's lyrics. PATKEY: 5ae2446bd5828c9e27deb3865118d9e783aa6e15
import_github()

In [46]:
# Extract all of Mike's lyrics. PATKEY: 5ae2446bd5828c9e27deb3865118d9e783aa6e15
Text = open("AllLyrics.txt", "r").read()
Vocabulary = ''.join([i for i in Text if not i.isdigit()]).replace("\n", " ").split(' ')

In [45]:
print(Vocabulary)

['Eighteen', 'years', 'eighteen', 'years', 'She', 'got', 'one', 'of', 'your', 'kids', 'got', 'you', 'for', 'eighteen', 'years', 'I', 'know', 'somebody', 'paying', 'child', 'support', 'for', 'one', 'of', 'his', 'kids', 'His', 'baby', 'mama', 'car', 'and', 'crib', 'is', 'bigger', 'than', 'his', 'You', 'will', 'see', 'him', 'on', 'TV', 'any', 'given', 'Sunday', 'Win', 'the', 'Super', 'Bowl', 'and', 'drive', 'off', 'in', 'a', 'Hyundai', 'She', 'was', 'supposed', 'to', 'buy', 'your', 'shorty', 'Tyco', 'with', 'your', 'money', 'She', 'went', 'to', 'the', 'doctor', 'got', 'lipo', 'with', 'your', 'money', 'She', 'walking', 'around', 'looking', 'like', 'Michael', 'with', 'your', 'money', 'Shouldve', 'got', 'that', 'insured', 'Geico', 'for', 'your', 'money', 'If', 'you', 'aint', 'no', 'punk', 'Holla', 'We', 'want', 'prenup', 'We', 'want', 'prenup', 'Yeah', 'Its', 'something', 'that', 'you', 'need', 'to', 'have', 'Cause', 'when', 'she', 'leave', 'your', 'bum', 'she', 'gon', 'leave', 'with', 'half

In [ ]:
# Generate text

In [ ]:
def line_generator(Vocabulary):
    index = 1
    chain = {}
    count = random.randint(6, 12)
    
    for word in Vocabulary[index:]:
        key = Vocabulary[index-1]
        if key in chain:
            chain[key].append(word)
        else:
            chain[key] = [word]
        index += 1
        
    word1 = random.choice(list(chain.keys()))
    line = word1.capitalize()

    while len(line.split(' ')) < count:
        word2 = random.choice(chain[word1])
        word1 = word2
        line += ' ' + word2
    return line

In [ ]:
line_generator(Vocabulary)

'Egotistic He nose \'cause you Might still tryin\' to me "Let\'s just'

In [ ]:
# Insert rhyme functions

In [ ]:
def reverse_syllable_extract(text):
    sy_form = []
    characters = [char for char in text]
    sylls = ['a', 'e', 'i', 'o', 'u']
    for x in characters:
        if x in sylls:
            sy_form.append(x)
    sy_form.reverse()
    return sy_form

def rev_syllable_stop_count(text1, text2):
    count = True 
    i = 0
    counter = 0
    syll1 = reverse_syllable_extract(text1)
    syll2 = reverse_syllable_extract(text2)
    while count == True:
        if i < min(len(syll1), len(syll2)) and syll1[i] == syll2[i]:
            counter += 1
            i += 1
        else:
            count = False
    return counter

def next_line_stop_count(start_line, lines):
    sy_lines = []
    for i in lines:
        sy_lines.append(rev_syllable_stop_count(start_line, i))
    choice = sy_lines[0]
    count = 0
    for i in range(len(sy_lines)):
        if sy_lines[i] > choice:
            choice = sy_lines[i]
    return lines[sy_lines.index(choice)]

In [ ]:
# Now move to generate a 20 line rap

In [ ]:
# Will generate 1000 lines using line_generator then from first generated line pick next best line by rhyme 19 times

In [ ]:
def generate_rap(Vocabulary):
    start_line = line_generator(Vocabulary)
    all_other_lines = [line_generator(Vocabulary) for i in range(999)]
    rap = [start_line]
    
    for i in range (19):
        next_line = next_line_stop_count(rap[len(rap) - 1], all_other_lines)
        all_other_lines.remove(next_line)
        rap.append(next_line)
    return rap

In [ ]:
generate_rap(Vocabulary)

["Superhuman Innovative and watch me is gon'",
 'Pastel I love to get in for my',
 'Flirt Im about my city lights cause strobe lights go',
 'Judge Luda landed in first Im allergic to tell it so',
 'Entourage, if it Yo, give it now',
 'Save all comes once in session,',
 'Christ They actors ball like nitro And',
 '"totally with you Be good lace these ninjas hot Hair so',
 'Reach for the casualty of my family dog',
 "Fetish, I'ma go to do anything It only",
 "Faint Plus pay Pink Friday and frick you cool now ain't got",
 'Late" And the risin sun on Just keep on a standin\' ovation',
 "Prove that spark to me Loving ain't no",
 "Monotone Sometimes your conscience That's why he just imitating So",
 "Tooth smile in this movie in Shady's cute I just rappin God",
 'Loud, Say it up Saying "Don\'t',
 'Denim be a poop grab your biggest fan This world',
 'Wildest You begged for the burner in line She said I got',
 "Thirst and you can reminisce over my pants Tryin' to",
 "From, and Nicki, it's goin'

In [ ]:
# I've left it as a list with strings as the lines, we can have fun improving things if we think this is a good start

In [ ]:
# The above has taken ages to load as has made 1000 lines then compared each one to try and find the best next!
# Also the vocab is much longer now